In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.datasets import dump_svmlight_file
from rdkit.Chem import DataStructs
import numpy as np

In [2]:
with open('acd_logd_100.smiles', 'r') as file_in:
    dataset = np.zeros((0, ), dtype=np.int8)
    count = 0
    ys = []
    fps = []
    for line in file_in:
        count+=1
        if count == 1: continue

        (x, y) = line.split('\t')
        ys.append(y)
        mol = Chem.MolFromSmiles(x)
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
        fps.append(fp)
        
    xs = np.array(fps)

In [3]:
spark = SparkSession.builder.appName("SimpleApp").getOrCreate()

# Load and parse the data file, converting it to a DataFrame.
data = spark.read.format("libsvm").load("test.dat")

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="indexedFeatures")

# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, rf])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

rfModel = model.stages[1]
print(rfModel)  # summary only

spark.stop()

NameError: name 'SparkSession' is not defined